In [3]:
import os
import os.path as osp
import argparse
from datetime import date
import json
import random
import time
from pathlib import Path
import numpy as np
import numpy.linalg as LA
import torch.linalg
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import csv

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader


import pandas as pd
import glm


In [4]:
path = "/home/kmuvcl/source/oldCuTi/CuTi/matterport/mp3d_planercnn_json/cached_set_test.json"
root = "/home/kmuvcl/source/oldCuTi/CuTi/matterport/"

In [5]:
with open(path) as f:
        split = json.load(f)

dset = split

In [6]:
def _sqrt_positive_part(x: torch.Tensor) -> torch.Tensor:
    """
    Returns torch.sqrt(torch.max(0, x))
    but with a zero subgradient where x is 0.
    """
    ret = torch.zeros_like(x)
    positive_mask = x > 0
    ret[positive_mask] = torch.sqrt(x[positive_mask])
    return ret


def matrix_to_quaternion(matrix: torch.Tensor) -> torch.Tensor:
    """
    Convert rotations given as rotation matrices to quaternions.

    Args:
        matrix: Rotation matrices as tensor of shape (..., 3, 3).

    Returns:
        quaternions with real part first, as tensor of shape (..., 4).
    """
    if matrix.size(-1) != 3 or matrix.size(-2) != 3:
        raise ValueError(f"Invalid rotation matrix shape {matrix.shape}.")

    batch_dim = matrix.shape[:-2]
    m00, m01, m02, m10, m11, m12, m20, m21, m22 = torch.unbind(
        matrix.reshape(batch_dim + (9,)), dim=-1
    )

    q_abs = _sqrt_positive_part(
        torch.stack(
            [
                1.0 + m00 + m11 + m22,
                1.0 + m00 - m11 - m22,
                1.0 - m00 + m11 - m22,
                1.0 - m00 - m11 + m22,
            ],
            dim=-1,
        )
    )

    # we produce the desired quaternion multiplied by each of r, i, j, k
    quat_by_rijk = torch.stack(
        [
            # pyre-fixme[58]: `**` is not supported for operand types `Tensor` and
            #  `int`.
            torch.stack([q_abs[..., 0] ** 2, m21 - m12, m02 - m20, m10 - m01], dim=-1),
            # pyre-fixme[58]: `**` is not supported for operand types `Tensor` and
            #  `int`.
            torch.stack([m21 - m12, q_abs[..., 1] ** 2, m10 + m01, m02 + m20], dim=-1),
            # pyre-fixme[58]: `**` is not supported for operand types `Tensor` and
            #  `int`.
            torch.stack([m02 - m20, m10 + m01, q_abs[..., 2] ** 2, m12 + m21], dim=-1),
            # pyre-fixme[58]: `**` is not supported for operand types `Tensor` and
            #  `int`.
            torch.stack([m10 - m01, m20 + m02, m21 + m12, q_abs[..., 3] ** 2], dim=-1),
        ],
        dim=-2,
    )

    # We floor here at 0.1 but the exact level is not important; if q_abs is small,
    # the candidate won't be picked.
    flr = torch.tensor(0.1).to(dtype=q_abs.dtype, device=q_abs.device)
    quat_candidates = quat_by_rijk / (2.0 * q_abs[..., None].max(flr))

    # if not for numerical problems, quat_candidates[i] should be same (up to a sign),
    # forall i; we pick the best-conditioned one (with the largest denominator)

    return quat_candidates[
        F.one_hot(q_abs.argmax(dim=-1), num_classes=4) > 0.5, :
    ].reshape(batch_dim + (4,))

def quaternion_to_matrix(quaternions: torch.Tensor) -> torch.Tensor:
    """
    Convert rotations given as quaternions to rotation matrices.

    Args:
        quaternions: quaternions with real part first,
            as tensor of shape (..., 4).

    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).
    """
    r, i, j, k = torch.unbind(quaternions, -1)
    # pyre-fixme[58]: `/` is not supported for operand types `float` and `Tensor`.
    two_s = 2.0 / (quaternions * quaternions).sum(-1)

    o = torch.stack(
        (
            1 - two_s * (j * j + k * k),
            two_s * (i * j - k * r),
            two_s * (i * k + j * r),
            two_s * (i * j + k * r),
            1 - two_s * (i * i + k * k),
            two_s * (j * k - i * r),
            two_s * (i * k - j * r),
            two_s * (j * k + i * r),
            1 - two_s * (i * i + j * j),
        ),
        -1,
    )
    return o.reshape(quaternions.shape[:-1] + (3, 3))

In [17]:
dset['data'][0]['1']

{'file_name': '/Pool1/users/jinlinyi/dataset/mp3d_rpnet_v4_sep20/rgb/2t7WUuJeko7/0_0_40.png',
 'image_id': '2t7WUuJeko7_0_0_40',
 'height': 480,
 'width': 640,
 'camera': {'position': [3.3564486204767814,
   1.576284381380687,
   -0.15686175964552806],
  'rotation': [-0.7689894865757656,
   0.07404526573629436,
   0.6320354113969981,
   0.06085808819055669]},
 'annotations': [{'id': 13,
   'image_id': '2t7WUuJeko7_0_0_40',
   'category_id': 0,
   'iscrowd': 0,
   'area': 7560,
   'bbox': [195.0, 35.0, 72.0, 121.0],
   'segmentation': [[254.0,
     155.5,
     248.0,
     152.5,
     237.0,
     155.5,
     208.0,
     152.5,
     202.5,
     145.0,
     198.5,
     134.0,
     194.5,
     86.0,
     194.5,
     46.0,
     199.0,
     42.5,
     204.5,
     42.0,
     202.5,
     44.0,
     205.5,
     45.0,
     205.0,
     50.5,
     213.0,
     38.5,
     238.0,
     34.5,
     247.0,
     38.5,
     260.5,
     53.0,
     266.5,
     137.0,
     262.5,
     148.0,
     254.0,
     1

In [18]:
file_name0 = []
file_name1 = []
for i in tqdm(range(len(dset['data']))):
    file_name0.append(dset['data'][i]['0']['file_name'])
    file_name1.append(dset['data'][i]['1']['file_name'])


100%|██████████| 7996/7996 [00:00<00:00, 677706.36it/s]


In [21]:
len(file_name0)

7996

In [22]:
import pandas as pd

In [24]:
rot = pd.read_csv("/home/kmuvcl/source/CuTi/logs/runs/2023-11-27/13-27-23/output2/gt_rotation_magnitude_vs_error.csv")

In [31]:
rot[' error'].idxmin()

6681

In [32]:
rot[' error'][6681]

0.03956

In [33]:
print(file_name0[6681])
print(file_name1[6681])

/Pool1/users/jinlinyi/dataset/mp3d_rpnet_v4_sep20/rgb/yqstnuAEVhm/0_4_10.png
/Pool1/users/jinlinyi/dataset/mp3d_rpnet_v4_sep20/rgb/yqstnuAEVhm/0_4_8.png


In [53]:
rot[rot[' error'] > 30]

,gt,pred,error
1,153.70143,11.45018,165.14919
3,55.30918,20.54430,75.85343
6,48.11283,3.39509,51.50239
9,124.66215,18.86072,143.52280
11,130.57205,7.50508,138.07098
...,...,...,...
7987,62.90802,78.22734,141.11896
7989,81.88999,2.39461,84.27349
7991,119.15276,4.42763,114.73507
7992,127.06060,28.24598,98.81540


In [52]:
print(file_name0[6681])
print(file_name1[6681])

/Pool1/users/jinlinyi/dataset/mp3d_rpnet_v4_sep20/rgb/yqstnuAEVhm/0_4_10.png
/Pool1/users/jinlinyi/dataset/mp3d_rpnet_v4_sep20/rgb/yqstnuAEVhm/0_4_8.png


In [56]:
print(file_name0[11])
print(file_name1[11])

/Pool1/users/jinlinyi/dataset/mp3d_rpnet_v4_sep20/rgb/2t7WUuJeko7/0_0_15.png
/Pool1/users/jinlinyi/dataset/mp3d_rpnet_v4_sep20/rgb/2t7WUuJeko7/0_0_24.png
